In [ ]:
# Data Shaping
import pandas as pd
import datetime as dt

# Logging
import logging

# API Calls
import requests
from requests.structures import CaseInsensitiveDict

# File Handling and Export
import json
import os
import hvac

# Error Handling
import traceback
import inspect

In [ ]:
config = {
    "job_title": "datto_rmm_api_devices",
    "logs_dir": "d:/logs",
    "job_timestamp": dt.datetime.utcnow().strftime("%Y_%m_%d_%H%M%S")
}

In [ ]:
logger = logging.getLogger(config["job_title"])

logger.setLevel(logging.DEBUG)

SUCCESS = 15
logging.addLevelName(SUCCESS, 'SUCCESS')

filename = f'{config["logs_dir"]}/{config["job_title"]}/{config["job_title"]}_{dt.datetime.utcnow().strftime("%Y_%m_%d")}.log'

fileHandler = logging.FileHandler(filename, mode="a")  #'a' for append you can use 'w' for write

formatter = logging.Formatter(
    "%(asctime)s,%(levelname)s,[%(lineno)d],%(funcName)s(),%(message)s",
    "%Y-%m-%d %H:%M:%S")

fileHandler.setFormatter(formatter)
logger.addHandler(fileHandler)

In [ ]:
result_ok = {
    "status_code": 200,
    "task_title": config["job_title"],
    "function": inspect.stack()[0][3],
    "message": "Success"
}

result_error = {
    "job_title": config["job_title"],
    "status_code": 500,
    "function": inspect.stack()[0][3],
    "message": traceback.format_exc()
}

In [ ]:
def read_secret(path):
    ca_path = os.environ.get("SSL_CERT_FILE")
    
    client = hvac.Client(verify=ca_path)
    # resp = client.read(path) 
    print(client.is_authenticated())
    # return resp
    resp = client.kv.v2.read_secret( path=f'/{path}')
    secret = resp['data']['data']

    return secret

In [ ]:
read_secret("secret/localhost/minio/flask")

In [ ]:
def __create_token(config):
    try:
        # call token api url
        token_uri = f'{config["base_uri"]}/auth/oauth/token'

        # construct header
        headers = CaseInsensitiveDict()
        headers["Content-Type"] = "application/x-www-form-urlencoded"

        # construct req body
        data = CaseInsensitiveDict()
        data["grant_type"] = "password"
        data["username"] = config["api_key"]
        data["password"] = config["api_secret"]

        # request content response
        resp = requests.post(token_uri, headers=headers, data=data, auth=("public-client", "public"))
        content = resp.content.decode("utf-8")
        c_dict = json.loads(content)

        return {
            "access_token": c_dict["access_token"],
            "result": result_ok
        }

    except Exception as e:
        logger.error(e)
        return {
            "result": result_error
        }


In [ ]:
def __api_pagination(config, url):
    try:
        # construct header
        headers = CaseInsensitiveDict()
        headers["Authorization"] = f'Bearer {config["access_token"]}'
        headers["Content-Type"] = "application/json"

        # construct req body
        data = ''

        print(f'Request URL: {url}')

        resp = requests.get(url, headers=headers, data=data)
        content = resp.content.decode('utf-8')
        c_dict = json.loads(content)

        return {
            "data": c_dict,
            "result": result_ok
        }

    except Exception as e:
        logger.error(e)
        return {
            "result": result_error
        }

In [ ]:
def create_devices_dataframe(config):
    try:
        # Create Devices DataFrame
        request_url = f'{config["base_uri"]}/api/v2/account/devices'
        data = __api_pagination(config, request_url)
        c_dict = data["data"]

        # iterate and combine remaining pages
        df = pd.DataFrame(c_dict["devices"])
        while c_dict["pageDetails"]["nextPageUrl"]:
            next_page = c_dict["pageDetails"]["nextPageUrl"]
            data = __api_pagination(config, next_page)
            c_dict = data["data"]
            print(c_dict["pageDetails"]["nextPageUrl"])
            df_current_page = pd.DataFrame(c_dict["devices"])
            df = pd.concat([df, df_current_page], ignore_index=False)

        df['lastAuditDate'] = pd.to_datetime(df['lastAuditDate'], unit='ms', errors='coerce')

        df['lastSeen'] = pd.to_datetime(df['lastSeen'], unit='ms', errors='coerce')

        df['creationDate'] = pd.to_datetime(df['creationDate'], unit='ms', errors='coerce')

        df['lastReboot'] = pd.to_datetime(df['lastReboot'], unit='ms', errors='coerce')

        logger.log(SUCCESS, "dataframe created")

        return {
            "data": c_dict,
            "result": result_ok
        }

    except Exception as e:
        logger.error(e)
        return {
            "result": result_error
        }



In [ ]:
config.update(read_secret(mount_point="api",
                          path="datto_rmm/systems_example_co"))

config.update(__create_token(config))

In [ ]:
try:
    os.mkdir(f'{config["logs_dir"]}/{config["title"]}')
except:
    pass

In [ ]:
result = create_devices_dataframe(config)

In [ ]:
result["result"]

In [ ]:
import luigi
df  = pd.DataFrame(result["data"])